In [3]:
#Import libraries
import pandas as pd

In [2]:
!pip install pandas

     |████████████████████████████████| 10.4MB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 12.4MB/s eta 0:00:01


In [2]:
#!pip install TextBlob

In [4]:
dirPath = r"\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
reviewPath = r"cleanData_reviews_Musical_Instruments_5.json.gz"
path = dirPath + "\\" + reviewPath
path = "preprocessed_reviews_Musical_Instruments_5.pkl.gz"
path = "preprocessed_reviews_Tools_and_Home_Improvement_5.pkl.gz"

In [5]:
data = pd.read_pickle(path)
data.shape

(123707, 7)

In [6]:
negativeReviews = data.loc[data['trueRating'] == 0, :]
negativeReviews.shape[0]


10105

In [7]:
#data = data.iloc[0:size*5, :]

In [8]:
negativeReviews.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123477,134239,B00JPBDL9W,I have converted three-quarters of my home to ...,"[0, 0]",2.0,0.0,i have convert three quarter of my home to lea...
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [9]:
#Pad more negative reviews for better training purposes
testSample = pd.concat([data, negativeReviews])
#testSample = data
#testSample.reset_index(drop=True, inplace=True)
testSample.tail(1)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [10]:
import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

133812

In [13]:
#!pip install sklearn


In [14]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [15]:
num_words = 15000

In [16]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [17]:
X = largeText
y = testSample['trueRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [18]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100359, 33453, 100359, 33453)

In [15]:
#x_train_tokens[1], x_train_text[1]

In [19]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [20]:
np.mean(num_tokens), np.max(num_tokens)


(111.61465339431442, 4959)

In [21]:
np.std(num_tokens)

133.85901020345165

In [22]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


379

In [23]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9588078797118346

In [24]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((100359, 379), (33453, 379))

In [25]:
for i in range(3, 4):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [26]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [27]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [28]:
x_train_text[1]

'very handy and easy to use, these arrive on time and in good condition, nice to have in shop when you need it you need it'

In [29]:
tokens_to_string(x_train_tokens[1])

'very handy and easy to use these arrive on time and in good condition nice to have in shop when you need it you need it'

In [30]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [33]:
#!pip install keras

In [38]:
index = len(x_train_pad)
index
len(y_train), len(y_test), len(y_train) + len(y_test)

(100359, 33453, 133812)

In [34]:
#abstract CNN
import modelCNNImplementation as modelCNN
import importlib
importlib.reload(modelCNN)

Using TensorFlow backend.


<module 'modelCNNImplementation' from '/home/toan/Study/NLP/modelCNNImplementation.py'>

In [36]:
import modelRNNImplementation as modelRNN
import importlib
importlib.reload(modelRNN)

<module 'modelRNNImplementation' from '/home/toan/Study/NLP/modelRNNImplementation.py'>

In [100]:
# Refactor here to run multiple simulations

cls_true = np.array(y_test)

numOfSimulations = 10

for eachRun in range (0, numOfSimulations):
    model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
    model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens, embedding_size=200)
    #custom_batch_size=32+eachRun*6
    #custom_batch_size=50+eachRun*6
    custom_batch_size=56
    print(f"batch size {custom_batch_size}")
    #train models
    model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=custom_batch_size)
    #model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=128)
    #predict results
    cnn_y_pred = model_cnn.predict(x=x_test_pad)
    cnn_y_pred = cnn_y_pred.T[0]
    #rnn_y_pred = model_rnn.predict(x=x_test_pad)
    #rnn_y_pred = rnn_y_pred.T[0]
    cnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in cnn_y_pred])
    #rnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in rnn_y_pred])
    #result matrix including duplicated results
    y_test.index
    y_test.values
    result = pd.DataFrame()
    result['Index'] = y_test.index
    result['TrueRating'] = y_test.values
    result['CNNRating'] = cnn_cls_pred
    #result['RNNRating'] = rnn_cls_pred
    # need to drop the duplicates due to negative padding
    result.drop_duplicates(subset="Index", inplace=True)
    #merge results  back
    df = testSample.loc[result.Index.to_list(), :]
    df = df.reset_index()
    df= df.drop_duplicates(subset="index")
    df.rename(columns={"index": "Index"}, inplace=True)
    modelResult = pd.merge(df, result, on="Index", how="inner")
    y_true = modelResult.TrueRating.to_list()
    cnn_pred = modelResult.CNNRating.to_list()
    #rnn_pred = modelResult.RNNRating.to_list() 
    report = dict()
    report['CNN'] = confusion_matrix(y_true, cnn_pred).ravel()
    #report['RNN'] = confusion_matrix(y_true, rnn_pred).ravel()
    y_true = np.array(y_true)
    reportCard = pd.DataFrame(report).T
    reportCard.columns = ['tn', 'fp', 'fn', 'tp']
    reportCard.loc['ActualRating'] = [len(np.where(y_true==0)[0]), 0, 0, len(np.where(y_true==1)[0])]
    reportCard.loc['custom_batch_size'] = custom_batch_size
    reportCard.to_csv(f"run{eachRun}_PaddingNegativeReviews_CNN.csv")
    del model_cnn
    del model_rnn  
    



    
    

batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 143us/step - loss: 0.2292 - accuracy: 0.9095 - val_loss: 0.1677 - val_accuracy: 0.9362
Epoch 2/3
95341/95341 [==============================] - 13s 141us/step - loss: 0.1256 - accuracy: 0.9524 - val_loss: 0.1406 - val_accuracy: 0.9440
Epoch 3/3
95341/95341 [==============================] - 14s 143us/step - loss: 0.0713 - accuracy: 0.9740 - val_loss: 0.1329 - val_accuracy: 0.9512
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 146us/step - loss: 0.2210 - accuracy: 0.9121 - val_loss: 0.1616 - val_accuracy: 0.9400
Epoch 2/3
95341/95341 [==============================] - 14s 144us/step - loss: 0.1255 - accuracy: 0.9518 - val_loss: 0.1370 - val_accuracy: 0.9456
Epoch 3/3
95341/95341 [==============================] - 14s 145us/step - loss: 0.0754 - accuracy: 0.9727 - val_loss: 0.1350 - val_accuracy: 0.9520
batch size 56


...

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.2257 - accuracy: 0.9099 - val_loss: 0.1639 - val_accuracy: 0.9346
Epoch 2/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.1247 - accuracy: 0.9525 - val_loss: 0.1439 - val_accuracy: 0.9458
Epoch 3/3
95341/95341 [==============================] - 13s 141us/step - loss: 0.0723 - accuracy: 0.9737 - val_loss: 0.1420 - val_accuracy: 0.9522
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.2258 - accuracy: 0.9104 - val_loss: 0.1806 - val_accuracy: 0.9227
Epoch 2/3
95341/95341 [==============================] - 13s 141us/step - loss: 0.1258 - accuracy: 0.9523 - val_loss: 0.1432 - val_accuracy: 0.9452
Epoch 3/3
95341/95341 [==============================] - 13s 142us/step - loss: 0.0751 - accuracy: 0.9730 - val_loss: 0.1320 - val_accuracy: 0.9582
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.2257 - accuracy: 0.9098 - val_loss: 0.1769 - val_accuracy: 0.9279
Epoch 2/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.1286 - accuracy: 0.9510 - val_loss: 0.1446 - val_accuracy: 0.9462
Epoch 3/3
95341/95341 [==============================] - 13s 141us/step - loss: 0.0784 - accuracy: 0.9718 - val_loss: 0.1546 - val_accuracy: 0.9438


In [35]:
model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 379, 200)          3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 378, 100)          40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 3,066,213
Trainable params: 3,066,213
Non-trainable params: 0
_________________________________________________________________


In [37]:
model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens, embedding_size=200)
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 379, 200)          3000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 379, 16)           10416     
_________________________________________________________________
gru_2 (GRU)                  (None, 379, 8)            600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 3,011,177
Trainable params: 3,011,177
Non-trainable params: 0
_________________________________________________________________


In [44]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=64)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 12s 130us/step - loss: 0.1246 - accuracy: 0.9531 - val_loss: 0.1597 - val_accuracy: 0.9370
Epoch 2/3
95341/95341 [==============================] - 12s 129us/step - loss: 0.0734 - accuracy: 0.9738 - val_loss: 0.1367 - val_accuracy: 0.9526
Epoch 3/3
95341/95341 [==============================] - 12s 130us/step - loss: 0.0422 - accuracy: 0.9848 - val_loss: 0.1658 - val_accuracy: 0.9504
CPU times: user 31.4 s, sys: 6.88 s, total: 38.3 s
Wall time: 37.1 s


In [42]:
#sess = tf.session(config=tf.ConfigProto(log_device_placement=True))
tf.test.gpu_device_name()


'/device:GPU:0'

In [45]:
%%time
model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=128)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 524s 5ms/step - loss: 0.2276 - accuracy: 0.9135 - val_loss: 0.1991 - val_accuracy: 0.9283
Epoch 2/3
95341/95341 [==============================] - 524s 5ms/step - loss: 0.1498 - accuracy: 0.9475 - val_loss: 0.1562 - val_accuracy: 0.9438
Epoch 3/3
95341/95341 [==============================] - 529s 6ms/step - loss: 0.0970 - accuracy: 0.9697 - val_loss: 0.1546 - val_accuracy: 0.9518
CPU times: user 46min 2s, sys: 1min 32s, total: 47min 35s
Wall time: 26min 17s


In [46]:
#%%time
cnn_result = model_cnn.evaluate(x_test_pad, y_test)

33453/33453 [==============================] - 1s 44us/step


In [47]:
#%%time
rnn_result = model_rnn.evaluate(x_test_pad, y_test)

33453/33453 [==============================] - 229s 7ms/step


In [48]:
print("Accuracy: {0:.2%}".format(rnn_result[1]))

Accuracy: 94.75%


In [49]:
print("Accuracy: {0:.2%}".format(cnn_result[1]))


Accuracy: 95.03%


In [50]:
%%time
cnn_y_pred = model_cnn.predict(x=x_test_pad)
cnn_y_pred = cnn_y_pred.T[0]

CPU times: user 1.21 s, sys: 168 ms, total: 1.38 s
Wall time: 1.07 s


In [51]:
%%time
rnn_y_pred = model_rnn.predict(x=x_test_pad)
rnn_y_pred = rnn_y_pred.T[0]

CPU times: user 7min 44s, sys: 20.3 s, total: 8min 4s
Wall time: 3min 51s


In [52]:
pd.DataFrame(cnn_y_pred).describe(), pd.DataFrame(rnn_y_pred).describe()

(                  0
 count  33453.000000
 mean       0.833978
 std        0.346778
 min        0.000003
 25%        0.979336
 50%        0.999923
 75%        0.999999
 max        1.000000,                   0
 count  33453.000000
 mean       0.856403
 std        0.312232
 min        0.008256
 25%        0.979231
 50%        0.993960
 75%        0.994908
 max        0.995259)

In [53]:
cnn_y_pred.shape, rnn_y_pred.shape

((33453,), (33453,))

In [54]:
cnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in cnn_y_pred])
rnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in rnn_y_pred])

In [55]:
len(cnn_cls_pred), len(rnn_cls_pred)

(33453, 33453)

In [56]:
cnn_accuracy = (cnn_cls_pred==y_test).mean()
rnn_accuracy = (rnn_cls_pred==y_test).mean()
cnn_accuracy, rnn_accuracy

(0.9502585717274983, 0.9475383373688459)

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
#Check for padding
tn, fp, fn, tp = confusion_matrix(y_test, cnn_cls_pred).ravel()
tn, fp, fn, tp

(4436, 648, 1016, 27353)

In [59]:
tn, fp, fn, tp = confusion_matrix(y_test, rnn_cls_pred).ravel()
tn, fp, fn, tp

(3979, 1105, 650, 27719)

In [60]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

(5084, 28369)

In [61]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

(15126, 85233)

In [62]:
cls_true = np.array(y_test)

In [63]:
y_test.index
y_test.values
result = pd.DataFrame()
result['Index'] = y_test.index
result['TrueRating'] = y_test.values
result['CNNRating'] = cnn_cls_pred
result['RNNRating'] = rnn_cls_pred

In [64]:
result.shape

(33453, 4)

In [65]:
# need to drop the duplicates due to negative padding
result.drop_duplicates(subset="Index", inplace=True)
result.shape

(32836, 4)

In [66]:
result.head(1)

,Index,TrueRating,CNNRating,RNNRating
0,96807,1.0,1.0,1.0


In [67]:
df = testSample.loc[result.Index.to_list(), :]
df = df.reset_index()
#A.drop_duplicates()
df= df.drop_duplicates(subset="index")
df.rename(columns={"index": "Index"}, inplace=True)
df.shape, df.columns

((32836, 8),
 Index(['Index', 'masterIndex', 'asin', 'reviewText', 'helpful', 'overall',
        'trueRating', 'CleanedJoin'],
       dtype='object'))

In [68]:
df.head(4)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...
3,43612,47267,B000LG7G0A,The shade was easy to cut and easy to install....,"[0, 0]",5.0,1.0,"the shade be easy to cut and easy to install, ..."


In [69]:
modelResult = pd.merge(df, result, on="Index", how="inner")
modelResult.head(3)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin,TrueRating,CNNRating,RNNRating
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...,1.0,1.0,1.0
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...,1.0,1.0,1.0
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...,1.0,1.0,1.0


In [72]:
#!pip install TextBlob

In [73]:
import TextBlobImplementation as TextBlob
import importlib
importlib.reload(TextBlob)

<module 'TextBlobImplementation' from '/home/toan/Study/NLP/TextBlobImplementation.py'>

In [74]:
polaritySample = modelResult.iloc[3, :]['CleanedJoin']

TextBlob.getPolarityFromText(polaritySample.split(','))

1.0

In [75]:
import TextPreProcessing as TextProcessing
import importlib
importlib.reload(TextProcessing)

<module 'TextPreProcessing' from '/home/toan/Study/NLP/TextPreProcessing.py'>

In [81]:
%%time
modelResult['simplifiedText'] = modelResult.reviewText.apply(TextProcessing.simplifiedTextParagraph)

CPU times: user 8.65 s, sys: 28 ms, total: 8.68 s
Wall time: 8.7 s


In [80]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/toan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [82]:
%%time
#not very accurate using stemming
#modelResult['textBlobRating'] = modelResult.CleanedJoin.str.split(',')
modelResult['textBlobRating'] = modelResult.simplifiedText.apply(TextBlob.getPolarityFromText)

CPU times: user 1min 3s, sys: 0 ns, total: 1min 3s
Wall time: 1min 3s


In [83]:
modelResult.head(3)

,Index,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin,TrueRating,CNNRating,RNNRating,simplifiedText,textBlobRating
0,96807,105069,B004TEIMVC,"If you're an advanced user, whether that be a ...","[3, 4]",5.0,1.0,if you be an advance user whether that be a ho...,1.0,1.0,1.0,[if you are an advanced user whether that be ...,1.0
1,120279,130698,B00DQ3F0YY,You really get your money worth with this flas...,"[0, 0]",5.0,1.0,you really get your money worth with this flas...,1.0,1.0,1.0,[you really get your money worth with this fla...,1.0
2,117496,127682,B00BB2MMNE,Simple and clear instructions; including wiri...,"[0, 0]",5.0,1.0,simple and clear instruction include wire the ...,1.0,1.0,1.0,[simple and clear instructions including wir...,1.0


In [84]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
cnn_pred = modelResult.CNNRating.to_list()
textBlob_pred = modelResult.textBlobRating.to_list()

In [85]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
rnn_pred = modelResult.RNNRating.to_list() 
textBlob_pred = modelResult.textBlobRating.to_list()

In [86]:
#report = {'CNN': [], 'RNN': None, 'TextBlob':None}
report = dict()

In [87]:
#tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
report['CNN'] = confusion_matrix(y_true, cnn_pred).ravel()
#tn, fp, fn, tp
report['CNN']

array([ 4060,   407,  1016, 27353])

In [88]:
#tn, fp, fn, tp = confusion_matrix(y_true, rnn_pred).ravel()
report['RNN'] = confusion_matrix(y_true, rnn_pred).ravel()
#tn, fp, fn, tp
report['RNN']

array([ 3626,   841,   650, 27719])

In [89]:
#tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
report['TextBlob'] = confusion_matrix(y_true, textBlob_pred).ravel()
#tn, fp, fn, tp

In [90]:
pd.options.display.max_colwidth = 50

#productsReviewData.cleanedText.head(2)

In [91]:
y_true = np.array(y_true)

In [92]:
reportCard = pd.DataFrame(report).T
reportCard.columns = ['tn', 'fp', 'fn', 'tp']

In [93]:
reportCard.loc['ActualRating'] = [len(np.where(y_true==0)[0]), 0, 0, len(np.where(y_true==1)[0])]
reportCard

,tn,fp,fn,tp
CNN,4060,407,1016,27353
RNN,3626,841,650,27719
TextBlob,1336,3131,2026,26343
ActualRating,4467,0,0,28369


In [94]:
reportCard.to_csv("2nd_PaddingNegativeReviews_RNN_CNN_TextBlob.csv")

## 133K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(3913, 554, 742, 27627)  CNN for negative reviews padding 
(3445, 1022, 923, 27446) RNN with negative reviews padding
(1142, 3325, 1782, 26587) textBlob with negative reviews padding 

Note: RNN 3*8 word vector with  batch size = 64

## no padding extra negative reviews

tn, fp, fn, tp

(1667, 916, 813, 27531)  CNN without negative reviews padding
(1034, 1549, 270, 28074) RNN without negative reviews padding
(671, 1912, 1767, 26577) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(509, 2074, 4175, 24169) Multinomial Naive Bayes # n-gram (1,2), num_words = 20000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000




## 10K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(174, 27, 44, 2214) CNN for negative reviews padding 
(113, 88, 34, 2224)  RNN with negative review padding
(50, 151, 112, 2146) textBlob with negative padding 

## no padding extra negative reviews

tn, fp, fn, tp

(31, 78, 37, 2227)  CNN without negative reviews padding
(3, 106, 2, 2262)   RNN without negative reviews padding
(27, 82, 106, 2158) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(12, 97, 232, 2032) Multinomial Naive Bayes # n-gram (1,2), num_words = 10000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000


